# Attack

https://github.com/AI-secure/GMI-Attack

attack the private model use public data

private model is the segmentation net trained on GTA5

In [2]:
from models.deeplabv3 import DeepLabV3
import torch

d:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


torch.Size([1, 3, 256, 512])
torch.Size([1, 19, 256, 512])
torch.Size([1, 256, 32, 64])


In [11]:
from models.gan import Generator
from models.gan import Discriminator


z = torch.randn(2,100).cuda()
print(z.size())
G = Generator(100).cuda()
fake = G(z)
print(fake.size())

torch.Size([2, 100])
torch.Size([2, 3, 256, 512])


In [12]:


D = Discriminator(3).cuda()
f_logit = D(fake)
print(f_logit.size())

torch.Size([754])
